In [11]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report, make_scorer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model



data = pd.read_csv('data.csv')
song_names = data['song name']
song_names
data = data.drop(columns=['song name'])


In [3]:
#data2 = data[data.genre != 'westcoastrap']
msfcc_names = []
for i in range(1, 21):
    msfcc_names.append('mfcc{0}'.format(i))
inputs = data[['genre']]
outputs = data[['zero crossing rate', 'spectral bandwidth', 'spectral centroid', 'chroma'] + msfcc_names]

In [4]:
# creating a dict file  
genre_dict = {'alternative': 0, 'westcoastrap': 1,
                 'edm': 2, 'jazz': 3, 'classical': 4} 

inputs.genre = [genre_dict[item] for item in inputs.genre]

X_train, X_test, y_train, y_test = train_test_split(outputs, inputs, test_size=0.2)

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


<h2>KNN-analysis</h2>

In [9]:
# K nearest neighbor testing

k = 10

model = neighbors.KNeighborsClassifier(k)

model.fit(X_train, y_train)

y_hat = model.predict(X_test)

print('accuracy: ', model.score(X_test, y_test))



scorerVar = make_scorer(f1_score, pos_label=1, average='micro')

scores = cross_val_score(model, X_train, 
                            y_train['genre'], scoring = scorerVar, cv = StratifiedKFold(5, True))

print('f1score estimate using mean')
print(scores.mean())

print("\n")

print('very confusing matrix')

label_ord = list(set(y_test['genre']))
Titanic_Conf_matrix = confusion_matrix(y_hat, y_test['genre'], labels=label_ord)
print(Titanic_Conf_matrix)
print(label_ord)

print(genre_dict)


accuracy:  0.7954545454545454
f1score estimate using mean
0.5490909090909091


very confusing matrix
[[20  5  1  0  0]
 [ 2  6  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  2  0]
 [ 0  0  0  1  7]]
[0, 1, 2, 3, 4]
{'alternative': 0, 'westcoastrap': 1, 'edm': 2, 'jazz': 3, 'classical': 4}


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
<h3>KDTree</h3>

In [12]:

#this code prints cluters of all songs that are similar
from sklearn.neighbors import KDTree

arr = np.array(outputs.values)

tree = KDTree(arr, leaf_size=4)

# r = 1.5 (means radius 1.5)
all_nn_indices  = tree.query_radius(arr, r=150)

all_nns = [[song_names[idx] for idx in nn_indices] for nn_indices in all_nn_indices]
for nns in all_nns:
    print(nns)

['Nujabes - Lady Brown.mp3', '02 Airplanes.m4a']
['Oh Wonder - Technicolour Beat.mp3', 'Ice Cube - You Know How We Do It (Official Video).mp3', 'Chet Baker - Tenderly.mp3']
["Eazy-E - It's On.mp3", "Compton's Most Wanted - Hood Took Me Under.mp3", 'Xzibit - X.mp3', 'Snoop doggy dogg - aint no fun (if the homies cant get none).mp3', 'flicker_porterRobinson.mp3', 'Miles Davis - Milestones (Audio).mp3', 'Coheed and Cambria Neverender.mp3']
['Arctic Monkeys - Teddy Picker (Official Video).mp3', 'WC - The Streets ft. Snoop Dogg, Nate Dogg.mp3', 'Dr. Dre - Still D.R.E. ft. Snoop Dogg.mp3', 'Stephen - Crossfire.mp3', '213 Groupie Luv.mp3']
['snoop doggy dogg - doggy dogg world.mp3', '2Pac - Skandalouz.mp3']
['OceanBlue_WRLD.mp3', 'Snoop Dogg - Midnight Love.mp3', '02 The Two Sides of Monsieur Valenti.m4a', '09 Bourgeois.m4a']
['snoop doggy dogg - doggy dogg world.mp3', 'Dr. Dre - Forgot About Dre ft. Eminem, Hittman.mp3']
['03 Crystalised.m4a', 'Ryo Fukui - It Could Happen To You.mp3', 'Kurup

<h2>Logistic Regression</h2>

In [6]:
#logistic regression testing
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)

y_hat = model.predict(X_test)

model.score(X_test, y_test)

scorerVar = make_scorer(f1_score, pos_label=1, average='micro')

scores = cross_val_score(model, X_train, 
                            y_train['genre'], scoring = scorerVar, cv = StratifiedKFold(5, True))
print('f1score estimate using mean')
print(scores.mean())

print("\n")

print('very confusing matrix')
Titanic_Conf_matrix = confusion_matrix(y_hat, y_test['genre'])
print(Titanic_Conf_matrix)


f1score estimate using mean
0.8017893217893217


very confusing matrix
[[20  0  1  0  0]
 [ 0 11  0  1  0]
 [ 1  0  0  0  0]
 [ 1  0  0  2  0]
 [ 0  0  0  0  7]]


/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
#polynimal expansion with logicist regression 
pf = PolynomialFeatures(degree=4)
pf.fit(X_train)
Phi = pf.transform(X_train)

model = linear_model.LogisticRegression()
model.fit(Phi, y_train)

y_hat1 = model.predict_proba(pf.transform(X_test))




model.score(pf.transform(X_test), y_test)



scorerVar = make_scorer(f1_score, pos_label=1, average='micro')

scores = cross_val_score(model, X_train, 
                            y_train['genre'], scoring = scorerVar, cv = StratifiedKFold(5, True))


print('f1score estimate using mean')
print(scores.mean())

print("\n")

print('very confusing matrix')
Titanic_Conf_matrix = confusion_matrix(y_hat, y_test['genre'])
print(Titanic_Conf_matrix)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


f1score estimate using mean
0.7601443001443002


very confusing matrix
[[20  0  1  0  0]
 [ 0 11  0  1  0]
 [ 1  0  0  0  0]
 [ 1  0  0  2  0]
 [ 0  0  0  0  7]]


<h2>Experimentation on percentage of each type each song was</h2>

In [8]:
print('perc_alt', 'perc_classical', 'perc_edm', 'perc_jazz', 'perc_rap')
for predicted_scores, idx in zip(y_hat1, X_test.index):
    # normalize percentages
    perc_alt = "%.2f" % ((predicted_scores[0] / sum(predicted_scores)) * 100)
    perc_classical = "%.2f" % ((predicted_scores[1] / sum(predicted_scores)) * 100)
    perc_edm = "%.2f" % ((predicted_scores[2] / sum(predicted_scores)) * 100)
    perc_jazz = "%.2f" % ((predicted_scores[3] / sum(predicted_scores)) * 100)
    perc_rap = "%.2f" % ((predicted_scores[4] / sum(predicted_scores)) * 100)
    
    print(song_names[idx])
    print(perc_alt, '    ', perc_classical, '       ', perc_edm, '    ', perc_jazz, '    ', perc_rap)
#order: 'alt', 'classical', 'edm', 'jazz'

perc_alt perc_classical perc_edm perc_jazz perc_rap
03 S.O.S. In Bel Air.m4a
100.00      0.00         0.00      0.00      0.00
The Dogg Pound - I Don't Like to dream About Getting Paid.mp3
0.00      100.00         0.00      0.00      0.00
05 All I Need.m4a
96.50      0.00         3.22      0.28      0.00
03 I Turn My Camera On.m4a
91.42      0.00         8.58      0.00      0.00
05 80's Comedown Machine.m4a
99.99      0.01         0.00      0.00      0.00
Radiohead - Paranoid Android.mp3
100.00      0.00         0.00      0.00      0.00
johann-sebastian-bachair-on-g-string-youtubemp3free.org.mp3
0.00      0.00         0.00      0.62      99.38
Beethoven - Symphony No.7 in A major op.92 - II, Allegretto.mp3
0.00      0.02         0.00      0.75      99.23
10 Night Time.m4a
85.09      0.00         0.00      14.91      0.00
05 The Delicate Place.m4a
100.00      0.00         0.00      0.00      0.00
01 The Beast and Dragon, Adored.m4a
100.00      0.00         0.00      0.00      0.00
01 En